In [1]:
import os


In [2]:
dataset_path="/kaggle/input/shoplifting-dataset/Shoplifting dataset"

In [3]:
import os
import random
import shutil

# --------------------------
# Paths
# --------------------------
dataset_path = "/kaggle/input/shoplifting-dataset/Shoplifting dataset"  # read-only input
TARGET_DIR = "/kaggle/working/Dataset_split"       # writable working directory

# --------------------------
# Settings
# --------------------------
CLASSES = ["Normal", "Shoplifting"]
TRAIN_RATIO = 0.8
random.seed(42)

# --------------------------
# Train/Test split
# --------------------------
for cls in CLASSES:
    src_cls_dir = os.path.join(dataset_path, cls)
    videos = os.listdir(src_cls_dir)
    random.shuffle(videos)

    split_idx = int(TRAIN_RATIO * len(videos))
    train_videos = videos[:split_idx]
    test_videos = videos[split_idx:]

    for split, split_videos in zip(["train", "test"], [train_videos, test_videos]):
        dst_dir = os.path.join(TARGET_DIR, split, cls)
        os.makedirs(dst_dir, exist_ok=True)

        for video in split_videos:
            shutil.copy(
                os.path.join(src_cls_dir, video),
                os.path.join(dst_dir, video)
            )

print("Video-level train/test split completed.")
print(f"Split dataset saved to: {TARGET_DIR}")


Video-level train/test split completed.
Split dataset saved to: /kaggle/working/Dataset_split


In [4]:
import os
import cv2

# --------------------------
# Paths for Kaggle
# --------------------------
SOURCE_DIR = "/kaggle/working/Dataset_split"  # train/test split folder
OUTPUT_DIR = "/kaggle/working/Frames_split"   # where frames will be saved

# --------------------------
# Class mapping
# --------------------------
class_to_target = {"Normal": 0, "Shoplifting": 1}

# --------------------------
# Settings
# --------------------------
FRAME_SKIP = 4       # save every 5th frame
IMG_SIZE = (224, 224)  # resize frames

# --------------------------
# Function to extract frames safely
# --------------------------
def extract_frames(video_path, target, output_dir, start_index):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % FRAME_SKIP == 0:
            # Resize frame
            frame = cv2.resize(frame, IMG_SIZE)

            # Decide folder based on target
            folder = "normal" if target == 0 else "abnormal"
            frame_dir = os.path.join(output_dir, folder)
            os.makedirs(frame_dir, exist_ok=True)

            # Save frame
            cv2.imwrite(os.path.join(frame_dir, f"frame_{start_index + saved_count}.jpg"), frame)
            saved_count += 1

        frame_count += 1

    cap.release()

# --------------------------
# Extract frames from all videos
# --------------------------
i = 0
for split in ["train", "test"]:
    for cls in ["Normal", "Shoplifting"]:
        src_dir = os.path.join(SOURCE_DIR, split, cls)
        target = class_to_target[cls]

        for video_file in os.listdir(src_dir):
            video_path = os.path.join(src_dir, video_file)
            extract_frames(video_path, target, os.path.join(OUTPUT_DIR, split), i)
            i += 1000  # unique numbering for each video

print("Frame extraction for train/test split completed.")
print(f"Frames saved to: {OUTPUT_DIR}")


Frame extraction for train/test split completed.
Frames saved to: /kaggle/working/Frames_split


In [5]:


import numpy as np
import pandas as pd
import tensorflow as tf

2026-01-17 19:12:02.207149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768677122.630403      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768677122.738089      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768677123.799554      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768677123.799596      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768677123.799599      55 computation_placer.cc:177] computation placer alr

In [6]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2

train_data = tf.keras.preprocessing.image_dataset_from_directory(
     directory="/kaggle/working/Frames_split/train",
    image_size=IMG_SIZE,
    label_mode="categorical",  # one-hot encoding
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory="/kaggle/working/Frames_split/test",
    image_size=IMG_SIZE,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 12124 files belonging to 2 classes.


I0000 00:00:1768677141.537670      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1768677141.541558      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 3123 files belonging to 2 classes.


In [7]:
classes=train_data.class_names
classes

['abnormal', 'normal']

In [8]:
# How many rounds should we get the model to look through the data?
NUM_EPOCHS = 50 #@param {type:"slider", min:10, max:100, step:10}

In [9]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.map(lambda x, y: (preprocess_input(x), y)).cache().prefetch(AUTOTUNE)
test_data = test_data.map(lambda x, y: (preprocess_input(x), y)).cache().prefetch(AUTOTUNE)


In [10]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow as tf

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False  # freeze base model layers

inputs = layers.Input(shape=input_shape)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(2, kernel_regularizer=regularizers.l2(0.001))(x)
outputs = layers.Activation("sigmoid", dtype=tf.float32)(x)

model = tf.keras.Model(inputs, outputs)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         2,562 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,921,874 (22.59 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [11]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=2) # stop

In [12]:
# Train the model
model.fit(train_data, epochs=50
          , validation_data=test_data,callbacks=[ early_stopping])

Epoch 1/50


I0000 00:00:1768677157.807441     854 service.cc:152] XLA service 0x7e5e18115db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768677157.807478     854 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1768677157.807483     854 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1768677160.396831     854 cuda_dnn.cc:529] Loaded cuDNN version 91002
2026-01-17 19:12:47.813076: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:12:47.950974: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:12:49.187111: E external/local_xl

  3/379 ━━━━━━━━━━━━━━━━━━━━ 17s 46ms/step - accuracy: 0.5104 - loss: 0.8356   

I0000 00:00:1768677176.195161     854 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


377/379 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5644 - loss: 0.7123

2026-01-17 19:13:18.809048: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:18.945736: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:20.224061: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:20.365162: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:20.506614: E external/local_xla/xla/stream_

379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.5648 - loss: 0.7121

2026-01-17 19:13:44.623796: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:44.758535: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:45.988628: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-17 19:13:46.129214: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


379/379 ━━━━━━━━━━━━━━━━━━━━ 81s 138ms/step - accuracy: 0.5650 - loss: 0.7120 - val_accuracy: 0.6334 - val_loss: 0.6712
Epoch 2/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.7554 - loss: 0.6013 - val_accuracy: 0.6436 - val_loss: 0.6570
Epoch 3/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.7728 - loss: 0.5482 - val_accuracy: 0.6459 - val_loss: 0.6510
Epoch 4/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.7907 - loss: 0.5097 - val_accuracy: 0.6526 - val_loss: 0.6472
Epoch 5/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.8030 - loss: 0.4829 - val_accuracy: 0.6567 - val_loss: 0.6445
Epoch 6/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.8102 - loss: 0.4638 - val_accuracy: 0.6596 - val_loss: 0.6419
Epoch 7/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.8186 - loss: 0.4467 - val_accuracy: 0.6647 - val_loss: 0.6394
Epoch 8/50
379/379 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.8241 - loss: 0.4328 - val_accura

In [15]:
model.save("classificationModel.h5")